# Message Classifier, Multi Class, Single Label
Gilbert François Duivesteijn (gilbert@deep-impact.ch)


<img src="images/dt010612.gif" width=800>

In [1]:
import os
import pickle as pkl
import re
import string

import matplotlib.pyplot as plt
import nltk
from nltk import tokenize
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from lib.utils import plot_confusion_matrix

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, BoundaryNorm
from sklearn import neighbors
import matplotlib.patches as mpatches

In [2]:
np.set_printoptions(precision=3, linewidth=100)
%matplotlib notebook

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gilbert/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/gilbert/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Library functions

In [4]:
stemmer = nltk.PorterStemmer()
analyzer = CountVectorizer().build_analyzer()


def stemmed(doc):
    return [stemmer.stem(w) for w in analyzer(doc)]


def no_stemmed(doc):
    return [w for w in analyzer(doc)]


def mask_integers(s):
    return re.sub(r'\d+', '_INT', s)


def mask_times(text):
    """
    Replaces times written like 12:50, 1PM, 4:15am, etc to _time token.
    :param    text    Input text
    :return           Output text with replaced times.
    """
    re_time1 = '\d{1,2}[:.]\d{2}(?:am|pm|AM|PM)'
    re_time2 = '\d{1,2}[:.]\d{2}'
    re_time3 = '\d{1,2}(?:am|pm|AM|PM)'
    rec_time = re.compile(re_time1 + '|' + re_time2 + '|' + re_time3)
    return re.sub(rec_time, '_TIME', text)


def mask_emojis(text):
    """
    Replaces all different emojis to _emoji token.
    :param    text    Input text
    :return           Output text with replaced emojis.    
    """
    re_icons = ':[a-z-_]*:'
    re_ldsd = '\<(.*?)\>'
    rec_icons = re.compile(re_icons + "|" + re_ldsd)
    return re.sub(rec_icons, '_EMOJI', text)


def mask_all(text):
    text = mask_times(text)
    text = mask_emojis(text)
    text = mask_integers(text)
    return text


def train_and_test(steps, X_train, X_test, y_train, y_test):
    """
    Trains and tests the pipeline with the given steps. 
    :param steps:       List of operations inside the pipeline.
    :param X_train:     Training data
    :param X_test:      Training labels
    :param y_train:     Testing data
    :param y_test:      Testing labels
    :return:            Trained model
    """
    pipeline = Pipeline(steps)
    folds = 10
    xval_score = cross_val_score(pipeline, X_train, y_train, cv=folds, n_jobs=-1)
    
    xv_min = np.min(xval_score)
    xv_max = np.max(xval_score)
    xv_mean = np.mean(xval_score)
    xv_std = np.std(xval_score)
    print('{} fold Cross Validation Score: <{:.2f}, {:.2f}>; µ={:.2f}'.format(folds, xv_min, xv_max, xv_mean))
    pipeline = pipeline.fit(X_train, y_train)
    print('Score on test set: {:.2f}'.format(pipeline.score(X_test, y_test)))
    return pipeline


def tag_message(pipeline, message):
    y_pred = pipeline.predict([message])[0]
    print('{:>20} | {}'.format(dict_classes[y_pred], message))

Challenges:
- Real data, not easy separable in different classes.
- Multi class classification, more difficult than binary classification.
- Small dataset, not a lot of samples to train and test.
- Number of samples are not equally divided over the classes.

| Class | Description                                       |
| ----- | ------------------------------------------------- |
| 1     | Too late, away during office hours or early leave |
| 2     | Holidays or scheduled free days                   |
| 3     | Home Office                                       |
| 4     | Medical appointment                               |
| 5     | Ill, without consulting a medical                 |
| 6     | Work related absence (at client, conference)      |
| 7     | In office announcement                            |
| 8     | Miscellanious                                     |

In [5]:
dict_classes = {
    1: 'late/early',
    2: 'holidays',
    3: 'home office',
    4: 'med app',
    5: 'ill',
    6: 'business',
    7: 'in office',
    8: 'miscellanious'
}

## Vector Space Model (VSM)

### Vectorizer

Before we can use the text messages to train a classifier, we have to transform text into numbers. 

In [6]:
documents = [
    "But I don’t want to go among mad people, Alice remarked.",
    "Oh, you can’t help that, said the Cat: we’re all mad here. I’m mad. You’re mad.",
    "How do you know I’m mad? said Alice.",
    "You must be, said the Cat, or you wouldn’t have come here."
]

vectorizer = CountVectorizer(stop_words='english')
# vectorizer = CountVectorizer(tokenizer=nltk.tokenize.word_tokenize, stop_words='english', strip_accents='unicode')
X = vectorizer.fit_transform(documents)
vectorizer.vocabulary_

{'don': 3,
 'want': 11,
 'mad': 6,
 'people': 8,
 'alice': 0,
 'remarked': 9,
 'oh': 7,
 'help': 4,
 'said': 10,
 'cat': 1,
 'know': 5,
 'wouldn': 12,
 'come': 2}

It creates vector for every document:

In [7]:
X.toarray()

array([[1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 3, 1, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]], dtype=int64)

When transforming a text with unknown words, like `cat` in this example, vector is empty.

In [8]:
documents2 = ["Cat: Where are you going?",
            "Alice: Which way should I go?"]
X12 = vectorizer.transform(documents2).toarray()
X12

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

### Intermezzo: Search engine 

In [9]:
# Document collection
D = X.toarray()

# query
q = vectorizer.transform(["is alice mad?"]).toarray()

# Do the search by computing the dot product
res = D * q

# Sort highest ranked documents and show only documents with a score > 0
res_ranked = np.sum(res, axis=1)
res_index_sorted = np.argsort(res_ranked)[::-1]
res_index_sorted_filtered = res_index_sorted[res_ranked[res_index_sorted] > 0]

# Print the search results
for index in res_index_sorted_filtered:
    print('[ score: {} ] {}'.format(res_ranked[index], documents[index]))

[ score: 3 ] Oh, you can’t help that, said the Cat: we’re all mad here. I’m mad. You’re mad.
[ score: 2 ] How do you know I’m mad? said Alice.
[ score: 2 ] But I don’t want to go among mad people, Alice remarked.


### Transformer

With a count vectorizer, every word gets counted with the same weight, making frequently occuring words too important. There are several ways to penalizing frequent occuring words and rewarding rare occuring words. The most well known method is called term frequency - inverse document frequency (TF-IDF). TfIdf is implemented in scikit-learn as `TfidfTransformer`. Another algoritm is Okapi-BM25. 

In [10]:
transformer = TfidfTransformer()
Xt = transformer.fit_transform(X)
Xt.toarray()

array([[0.352, 0.   , 0.   , 0.446, 0.   , 0.   , 0.285, 0.   , 0.446, 0.446, 0.   , 0.446, 0.   ],
       [0.   , 0.305, 0.   , 0.   , 0.386, 0.   , 0.74 , 0.386, 0.   , 0.   , 0.247, 0.   , 0.   ],
       [0.505, 0.   , 0.   , 0.   , 0.   , 0.641, 0.409, 0.   , 0.   , 0.   , 0.409, 0.   , 0.   ],
       [0.   , 0.453, 0.575, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.367, 0.   , 0.575]])

### Stopwords

In [11]:
for word in nltk.corpus.stopwords.words('english'):
    print('{}, '.format(word), end='')

i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, must

### Stemming

In [12]:
doc11 = 'computer computers computing computed'
print(no_stemmed(doc11))
print(stemmed(doc11))

['computer', 'computers', 'computing', 'computed']
['comput', 'comput', 'comput', 'comput']


## Load and prepare the data

In [13]:
df_cls_messages = pd.read_pickle('data/messages-cls.pkl')
df_cls_messages.head(10)

,ts,user,text,class,key
0,1.503303e+09,Balaemar,I have to pick up my car from the garage tomor...,1,1503303350U035FRUCY
1,1.503302e+09,Ragaenys,"I won’t be here tomorrow, one day vacation",2,1503301710U4A2FRAQ4
2,1.503296e+09,Myke,Missed connection in Zurich. Will be about 5-1...,1,1503296123U0MGNKETU
3,1.503260e+09,Drevyn,Enjoy!,8,1503259722U035B8PRU
4,1.503258e+09,Gaelralis,I am away for 2 weeks in iceland :flag-is:,2,1503258060U0HLAK1T6
5,1.503223e+09,Gonaleos,Monday is the first day in school for my son a...,1,1503222795U3W886EF4
6,1.503063e+09,Vaerys,Going to leave around 4:50 because I need to g...,1,1503063218U0PLNK7EK
7,1.503059e+09,Taengaeron,"I am away between 3-4 for private stuff, be ba...",1,1503059074U4C55CKGX
8,1.503058e+09,Baegar,I will move to HO. I will be in 2 different Te...,3,1503057526U035J4RG4
9,1.503057e+09,Eliar,Need to leave around 16:45. Will be available ...,1,1503056595U0A8NPHRD


In [14]:
# Remove all rows which have no annotation
samples = df_cls_messages.dropna()

# Convert the classification column to unsigned int, in case it is stored as string
samples['class'] = samples.loc[:, 'class'].astype(np.uint8).values

X = samples['text']
y = samples['class']

print('[.] Number of training samples: {}'.format(len(X)))

[.] Number of training samples: 1719


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

## Building a pipeline and training the first model

Scikit Learn offers a great way to combine the preprocessing (vectorization, stemming, stopword removal, etc) and training/predicting by building a pipeline. Let's see how that works...

More information on pipelines, look at this nice blog post:

https://buhrmann.github.io/sklearn-pipelines.html

In [ ]:
steps = [('vectorizer', CountVectorizer()),
         ('classifier', LinearSVC(random_state=1))]
pipeline1 = train_and_test(steps, X_train, X_test, y_train, y_test)

In [ ]:
y_pred = pipeline1.predict(X_test)
cm = confusion_matrix(y_pred=y_pred, y_true=y_test)
plt.figure()
plot_confusion_matrix(cm, classes=dict_classes.values(), normalize=True);

Let's see what parameters are available and have been set by default:

In [ ]:
pipeline1.get_params()

All objects in the pipeline are stored in a dictonary. You can easily access them like any ordinary python dictionary. E.g. you want to transform only a document to a vector. This can be useful if you want to have the output of the preprocessing step for plotting or further analysis. 

In [ ]:
vectorizer = pipeline1.get_params()['vectorizer']
vectorizer.transform(['A new document']).toarray()

Let's add a TF-IDF transformer that suppress the weight of common words and make special words more important.

In [ ]:
steps = [('vectorizer', CountVectorizer()),
         ('tfidf', TfidfTransformer()),
         ('classifier', LinearSVC(random_state=1))]
pipeline2 = train_and_test(steps, X_train, X_test, y_train, y_test)

In [ ]:
steps = [('vectorizer', CountVectorizer(analyzer=stemmed)),
         ('tfidf', TfidfTransformer()),
         ('classifier', LinearSVC(random_state=1))]
pipeline3 = train_and_test(steps, X_train, X_test, y_train, y_test)

In [ ]:
steps = [('vectorizer', CountVectorizer(analyzer=stemmed, 
                                        stop_words='english')),
        ('tfidf', TfidfTransformer()),
        ('classifier', LinearSVC(random_state=1))]
pipeline4 = train_and_test(steps, X_train, X_test, y_train, y_test)

In [ ]:
steps = [('vectorizer', CountVectorizer(ngram_range=(1, 2))),
         ('tfidf', TfidfTransformer()),
         ('classifier', LinearSVC(random_state=1))]
pipeline5 = train_and_test(steps, X_train, X_test, y_train, y_test)

In [ ]:
steps = [('vectorizer', CountVectorizer(ngram_range=(1, 3))),
         ('tfidf', TfidfTransformer()),
         ('classifier', LinearSVC(random_state=1))]
pipeline6 = train_and_test(steps, X_train, X_test, y_train, y_test)

In [ ]:
steps = [('vectorizer', CountVectorizer(preprocessor=mask_all, 
                                        analyzer=stemmed, 
                                        stop_words='english', 
                                        ngram_range=(1, 3))),
         ('tfidf', TfidfTransformer()),
         ('classifier', LinearSVC(random_state=1))]
pipeline7 = train_and_test(steps, X_train, X_test, y_train, y_test)

## One vs Rest Classifiers

Let's train 8 classifiers: One vs Rest

In [ ]:
enc = OneHotEncoder()
ym_train = enc.fit_transform(np.array(y_train.tolist()).reshape(-1,1))
ym_test = enc.transform(np.array(y_test.tolist()).reshape(-1,1))

steps = [('vectorizer', CountVectorizer(analyzer=stemmed, 
                                        stop_words='english', 
                                        preprocessor=mask_all)),
         ('transformer', TfidfTransformer()),
         ('classifier', OneVsRestClassifier(LinearSVC(random_state=1, multi_class='ovr')))]
pipeline8 = train_and_test(steps, X_train, X_test, ym_train, ym_test)

In [ ]:
steps = [('vectorizer', CountVectorizer(ngram_range=(1,3), 
                                        analyzer=stemmed, 
                                        stop_words='english', 
                                        preprocessor=mask_all)),
         ('transformer', TfidfTransformer()),
         ('classifier', OneVsOneClassifier(LinearSVC(random_state=1, multi_class='ovr')))]
pipeline9 = train_and_test(steps, X_train, X_test, y_train, y_test)

In [ ]:
y_pred = pipeline9.predict(X_test)

print(classification_report(y_pred=y_pred, y_true=y_test))

In [ ]:
cm = confusion_matrix(y_pred=y_pred, y_true=y_test)
plt.figure()
plot_confusion_matrix(cm, classes=dict_classes.values(), normalize=True);

In [ ]:
%%time

steps = [('vectorizer', CountVectorizer()),
         ('transformer', TfidfTransformer()),
         ('classifier', LinearSVC(random_state=1))]
pipeline10 = Pipeline(steps)

params = {
    'vectorizer__tokenizer': [None, nltk.tokenize.word_tokenize],
    'vectorizer__analyzer': ['word', stemmed],
    'vectorizer__stop_words': [None, nltk.corpus.stopwords.words('english'), 'english'],
    'vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2), (1, 3)],
    'vectorizer__preprocessor': [None, mask_all, mask_integers, mask_times, mask_emojis]
    'classifier__C': np.logspace(-2, 2, 5),
    'classifier__gamma': np.logspace(-5, 3, 9)
}

gs = GridSearchCV(pipeline10, params, n_jobs=-1)
gs.fit(X_train, y_train)

print(gs.best_params_)
y_pred = gs.predict(X_test)
print(classification_report(y_pred=y_pred, y_true=y_test))
print('Score on the test set: {:.2f}'.format(gs.score(X_test, y_test)))

You can try a gridsearch with another classifier. But it is hard to beat Linear SVM.

In [19]:
%%time

steps = [('vectorizer', CountVectorizer()),
         ('transformer', TfidfTransformer()),
         ('classifier', RandomForestClassifier())]
pipeline11 = Pipeline(steps)

params = {
    'vectorizer__tokenizer': [None, nltk.tokenize.word_tokenize],
    'vectorizer__analyzer': ['word', stemmed],
    'vectorizer__stop_words': [None, nltk.corpus.stopwords.words('english'), 'english'],
    'vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2), (1, 3)],
    'vectorizer__preprocessor': [None, mask_all, mask_integers, mask_times, mask_emojis]
    'classifier__n_estimators': [128, 512, 2048],
    'classifier__bootstrap': [True, False],
    'classifier__criterion': ['gini', 'entropy'], 
    'classifier__max_depth': [2, 4, 8, None],
    'classifier__min_samples_leaf': [1, 3, 10], 
    'classifier__min_samples_split': [2, 8, 16]
}

gs = GridSearchCV(pipeline11, params, n_jobs=-1)
gs.fit(X_train, y_train)

print(gs.best_params_)
y_pred = gs.predict(X_test)
print(classification_report(y_pred=y_pred, y_true=y_test))
print('Score on the test set: {:.2f}'.format(gs.score(X_test, y_test)))

{'classifier__bootstrap': False, 'classifier__criterion': 'gini', 'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 8, 'classifier__n_estimators': 2048}
             precision    recall  f1-score   support

          1       0.77      0.88      0.82       100
          2       0.74      0.74      0.74        35
          3       0.82      0.89      0.85        95
          4       0.93      0.54      0.68        26
          5       0.83      0.45      0.59        11
          6       0.88      0.70      0.78        30
          7       1.00      0.17      0.29         6
          8       0.84      0.90      0.87        41

avg / total       0.81      0.81      0.80       344

Score on the test set: 0.81
CPU times: user 14.8 s, sys: 277 ms, total: 15.1 s
Wall time: 1min 36s


In [ ]:
pipeline = gs

tag_message(pipeline, 'My alarm clock was not set properly. I come to the office asap.')
tag_message(pipeline, 'It is my scheduled day off, see you on Tuesday.')
tag_message(pipeline, 'Not feeling well today, I stay home and work from here.')
tag_message(pipeline, 'I work at home on Tuesday.')
tag_message(pipeline, 'This morning I have a meeting at SPS.')
tag_message(pipeline, 'I\'m off, see you tomorrow.')
tag_message(pipeline, 'get well soon!')
tag_message(pipeline, 'I\'m away for a long lunch between 12:00 and 15:30')
tag_message(pipeline, 'I\'ve an appointment at 12:00 at the physiotherapy.')